# Setup

In [ ]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [8]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback
from tensorflow.math import multiply
from math import log

In [2]:
# If running in colab, insert your wandb key here

import config
#Erlend
wandb.login(key=config.erlend_key)
# Hjalmar
#wandb.login(key=config.hjalmar_key)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [3]:
google_colab = False

if google_colab:
    import tensorflow as tf
    # Pring info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1tSZeYgiZh_gL6CKUYufNlb1C6X4vODY3"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2021_2022.csv')  
    df_read = pd.read_csv('2021_2022.csv')
else:
    file = "../data/processed_data/2021_2022.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,...,Underlying_82,Underlying_83,Underlying_84,Underlying_85,Underlying_86,Underlying_87,Underlying_88,Underlying_89,Underlying_90,R
0,704239,2021-05-17,2021-05-19,3062.000,4163.60,1100.0,2,3.785091,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
1,704240,2021-05-17,2021-05-19,2962.505,4163.60,1200.0,2,3.469667,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
2,704241,2021-05-17,2021-05-19,2862.600,4163.60,1300.0,2,3.202769,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
3,704242,2021-05-17,2021-05-19,2761.600,4163.60,1400.0,2,2.974000,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
4,704243,2021-05-17,2021-05-19,2661.600,4163.60,1500.0,2,2.775733,-9.88,61.51,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250170,4059504,2022-12-30,2025-12-19,6.950,3839.81,8400.0,1085,0.457120,-8.85,65.61,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
3250171,4059505,2022-12-30,2025-12-19,5.950,3839.81,8600.0,1085,0.446490,-8.85,65.61,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
3250172,4059506,2022-12-30,2025-12-19,5.450,3839.81,8800.0,1085,0.436342,-8.85,65.61,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
3250173,4059507,2022-12-30,2025-12-19,4.500,3839.81,9000.0,1085,0.426646,-8.85,65.61,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22


### Format input data

In [5]:
df = df_read

# Format settings
max_timesteps = 90
moneyness = False # Moneyness = True WIP
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]

# Create train, validation and test set split points
train_start = datetime(2021,1,1) 
val_start = train_start + relativedelta(months=11) 
test_start = val_start + relativedelta(months=1)
test_end = test_start + relativedelta(months=1)
train_start = str(train_start.date())
val_start = str(val_start.date())
test_start = str(test_start.date())
test_end = str(test_end.date())

# Split train and validation data
df_train = df[(df['Quote_date'] >= train_start) & ( df['Quote_date'] < val_start)]
df_val = df[(df['Quote_date'] >= val_start) & ( df['Quote_date'] < test_start)]

# Extract target values
train_y = (df_train['Price']/df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
val_y = (df_val['Price']/df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()

# If usining moneyness, extract strike
if moneyness:
    train_strike = df_train['Strike'].to_numpy()
    val_strike = df_val['Strike'].to_numpy()

# Convert dataframes to numpy arrays
train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0])
val_x[0] = underlying_scaler.transform(val_x[0])

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]
if moneyness:
    train_strike = train_strike[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps,1), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

Train shape: (1119088, 90, 1), (1119088, 4)
Val shape: (202710, 90, 1), (202710, 4)


# Model construction

In [6]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [9]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 testing : 21-22 data',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 100, 200, 400, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40, 60, 90]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. Lr_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: sj7qrj73
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/sj7qrj73


# Run hyperparameter search

In [10]:
#WIP
class MSE_LossCallback(Callback):
    def __init__(self, train_x, train_y, train_strike, val_x, val_y, val_strike):
        self.train_x = train_x
        self.train_y = train_y
        self.train_strike = train_strike
        self.val_x = val_x
        self.val_y = val_y
        self.val_strike = val_strike
    
    def on_epoch_end(self, epoch, logs={}):
        train_pred = self.model(train_x)
        val_pred = self.model(val_x)

        train_mse = reduce_mean(square(multiply(train_pred[:,0] - self.train_y, self.train_strike)))
        val_mse = reduce_mean(square(multiply(val_pred[:,0] - self.val_y, self.val_strike)))

        print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')


In [11]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [12]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        if moneyness:
            mse_callback = MSE_LossCallback(train_x, train_y, train_strike, val_x, val_y, val_strike)

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 3,
            callbacks = [early_stopping, wandb_callback, mse_callback] if moneyness else [early_stopping, wandb_callback],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

### Run full sweep

In [13]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 3)

wandb: Agent Starting Run: hg695247 with config:
wandb: 	Bn_momentum: 0.99
wandb: 	LSTM_layers: 3
wandb: 	LSTM_timesteps: 40
wandb: 	LSTM_units: 4
wandb: 	Lr: 0.0018337320988027908
wandb: 	Lr_decay: 0.9745621588432394
wandb: 	MLP_layers: 8
wandb: 	MLP_units: 200
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 4)            384         ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 8)            0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▅█
loss,█▅▁
val_loss,█▄▁
best_epoch,2
best_val_loss,44983.77344
epoch,2
loss,250239.51562
val_loss,44983.77344


wandb: Agent Starting Run: sycoq85t with config:
wandb: 	Bn_momentum: 0.1
wandb: 	LSTM_layers: 2
wandb: 	LSTM_timesteps: 10
wandb: 	LSTM_units: 8
wandb: 	Lr: 0.01476985774273403
wandb: 	Lr_decay: 0.9460006569287052
wandb: 	MLP_layers: 6
wandb: 	MLP_units: 100
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            864         ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▅█
loss,█▁▁
val_loss,▂▁█
best_epoch,1
best_val_loss,1259.05286
epoch,2
loss,556.02057
val_loss,2878.75


wandb: Agent Starting Run: xwygdh21 with config:
wandb: 	Bn_momentum: 0.1
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 60
wandb: 	LSTM_units: 8
wandb: 	Lr: 0.00023258658893221963
wandb: 	Lr_decay: 0.93807002334871
wandb: 	MLP_layers: 3
wandb: 	MLP_units: 100
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            1952        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

### Single run

In [ ]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': f'Moneyness. Train_start: {train_start}, val_start: {val_start}, test_start: {test_start}',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')